## Todos

model isn't exactly reproducible. having different results when i load it

As of 24 September my best uncertainty model according to recall@30 is registered in wandb in the project "final_model_p1_pre_uncertainty_tunning" and it is called "golden-oath-84"

The config is as follow:
- type: gru
- hidden_size = 100
- lr: 0.01
- layers = 1
- batch_size = 64
- dropout: 0.1
- epochs: 15


In this notebook i will create a pickle of this model to be used for later experiments

# Change working directory to parent

Assumes this script is currently placed in a child folder of the main project.

In [1]:
import os
cwd = os.getcwd()

# protection against running this cell multiple times
assert os.path.dirname(cwd).split('/')[-1] == 'master-thesis','Oops, directory already changed previously as indended. Ignoring...'

# change working directory (if assert passed)
new_cwd = os.path.dirname(cwd) # parent directory
os.chdir(new_cwd)

In [2]:
# show all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
model_name = 'golden-oath-84'

In [4]:
import os
import json

from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import train_one_epochV2, eval_model

from config import Settings; settings = Settings()

from mourga_variational.variational_rnn import VariationalRNN

import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np
import pandas as pd

import wandb

# Parameters

In [5]:
dataset_id = 'diag_only'

# Reproducibility

In [6]:
# Reproducibility
seed = settings.random_seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Create dataset

In [7]:
# hyperparameters of best model
grouping = 'ccs'
batch_size=64

In [8]:
dataset_folder = os.path.join(settings.data_base,settings.model_ready_dataset_folder,dataset_id)
print('dataset at',dataset_folder)

dataset at data/model_ready_dataset/diag_only


In [9]:
dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset.json'),grouping)

train_dataset = DiagnosesDataset(os.path.join(dataset_folder,'train_subset.json'),grouping)
val_dataset = DiagnosesDataset(os.path.join(dataset_folder,'val_subset.json'),grouping)
test_dataset = DiagnosesDataset(os.path.join(dataset_folder,'test_subset.json'),grouping)


len(train_dataset)
len(val_dataset)
len(test_dataset)


train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

5249

1125

1125

# Define model

## Hyperparameters

In [10]:
# remaining hyperparameters of best model
input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
hidden_size = 100
num_layers = 1
n_labels = input_size
rnn_type = 'GRU'

lr = 0.01
dropout = 0.1

n_labels = input_size
epochs = 15
criterion = torch.nn.BCEWithLogitsLoss()

# Train

In [10]:
model = VariationalRNN(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          n_labels=n_labels,
                          rnn_type=rnn_type,
                          dropouti=dropout,
                          dropoutw=dropout,
                          dropouto=dropout)
    
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

criterion = torch.nn.BCEWithLogitsLoss(reduction='none')

for epoch in range(1,epochs+1):
    loss = train_one_epochV2(model,train_dataloader,epoch,criterion,optimizer)
    print(loss)

0.1764651072578954
0.12757067531416655
0.12295396654007945
0.11954586398142235
0.11722483995969298
0.11559556014493269
0.114258379394566
0.11309377855736515
0.11156229401891243
0.1107749301454974
0.10957034932499977
0.10929878544833799
0.10889105903798439
0.10809960733514187
0.10777546004538013


# Eval

confirm it has good results

In [11]:
train_results = eval_model(model,train_dataloader,dataset, criterion, epoch, 'train_last')
val_results = eval_model(model,val_dataloader,dataset, criterion, epoch, 'validation')
test_results = eval_model(model,test_dataloader,dataset, criterion, epoch, 'test')
res = {'train_loss':train_results['loss'],
       'train_recall@30':train_results['last adm']['recall30']['mean'],
       'val_loss':val_results['loss'],
       'test_loss':test_results['loss'],
       'recall@30_test':test_results['last adm']['recall30']['mean'],
       'recall@10':val_results['last adm']['recall10']['mean'],
       'recall@20':val_results['last adm']['recall20']['mean'],
       'recall@30':val_results['last adm']['recall30']['mean'],
       'n_val_recall@30_last':val_results['last adm']['recall30']['n']
      }
res

/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  

{'train_loss': 0.10245790746035315,
 'train_recall@30': 0.7555214618777231,
 'val_loss': 0.11377182188298968,
 'test_loss': 0.11668441123432584,
 'recall@30_test': 0.7033773807941549,
 'recall@10': 0.438171500887679,
 'recall@20': 0.6061258800410522,
 'recall@30': 0.7159257749353868,
 'n_val_recall@30_last': 1125}

## Save model weights and parameters

In [12]:
model_folder = os.path.join(settings.data_base,settings.models_folder,model_name)

if not os.path.isdir(model_folder):
    os.mkdir(model_folder)

# save weights
weights_save_path = os.path.join(model_folder,"weights")

torch.save(model.state_dict(), 
           weights_save_path
          )
print('Model saved!')

Model saved!


## Save model hyperparameters

In [13]:
params = dict(input_size = input_size,
              hidden_size=hidden_size,
              num_layers=num_layers,
              n_labels=n_labels,
              rnn_type=rnn_type,
              dropouti=dropout,
              dropouto=dropout,
              dropoutw=dropout
             )


hypp_save_path = os.path.join(model_folder, 'hyper_parameters.json')

with open(hypp_save_path, "w") as f:
    json.dump(params, f)
    
print('Hyperparameters saved!')

Hyperparameters saved!


## Other parameters

In [14]:
other_params = dict(batch_size=batch_size,
                    grouping=grouping,
                    epochs=epochs
                   )

other_params_save_path = os.path.join(model_folder, 'other_params.json')

with open(other_params_save_path, "w") as f:
    json.dump(other_params, f)
    
print('Other parameters saved!')

Other parameters saved!


# Test it out

### Read weights and hyperparameters

In [15]:
#hyperparameters
with open(hypp_save_path,'r') as f:
    params_loaded = json.load(f)
    
# weights
weights = torch.load(weights_save_path)

### Create model and load weights

In [16]:
new_model = VariationalRNN(**params_loaded)
new_model.load_state_dict(torch.load(weights_save_path))

<All keys matched successfully>

### Evaluate it

In [17]:
eval_model(new_model,val_dataloader,dataset, criterion, epoch, 'validation')['last adm']

{'recall10': {'mean': 0.4445565745815882,
  'std': 0.2000251888844687,
  'n': 1125},
 'recall20': {'mean': 0.6144619831876157,
  'std': 0.19039169995584382,
  'n': 1125},
 'recall30': {'mean': 0.7224851838296337,
  'std': 0.17209051951758605,
  'n': 1125}}

# Performance doesn't match

For some reason, the performance isn't exactly the same for the loaded model. Maybe they are different in their parameters?

In [18]:
for p1, p2 in zip(model.parameters(), new_model.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        print('Models don\'t match')
print('Models match')

Models match


.... weird
